<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/9_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- mp.Manager().dict() 來更新(共享)進度 :

  - 子 Process 要更新進度 → 主 Process UI 要讀取 → 就必須用 可共享的物件。
  - 如果你不用 Manager().dict()，而是用普通 dict，那進度更新只會存在子行程自己的記憶體，主行程完全看不到。



**所以情境區分：**

1. Threading 版本：

  - thread 跟主程式共享同一個記憶體空間

  - 用普通 dict 就夠了

  - progress_pool、stop_pool 這樣寫就行

2. Multiprocessing 版本：

  - process 跟主程式分開記憶體

  - 必須用 Manager().dict() 或 multiprocessing.Value / Array,否則無法即時更新 UI

In [ ]:
import dash
from dash import dcc, html, Input, Output
import multiprocessing as mp
import time

# ----------------------------
# Manager 共用狀態
# ----------------------------
manager = mp.Manager()
progress = manager.dict()
process = None   # 全域只會有一個流程

step_names = ["撈取資料", "進行 Rule Check", "繪製圖片", "上拋資料"]

def default_progress():
    return {
        "current_index": -1,
        "step_status": ["pending"] * len(step_names),
        "step_times": [None] * len(step_names),
        "done": False,
        "error": False,
        "status": "idle"
    }

# ----------------------------
# 報告流程
# ----------------------------
def run_report(progress):
    try:
        progress["status"] = "running"
        start_time = time.time()

        # step1
        progress["current_index"] = 0
        progress["step_status"][0] = "running"
        time.sleep(2)
        progress["step_status"][0] = "success"

        # step2
        progress["current_index"] = 1
        progress["step_status"][1] = "running"
        time.sleep(2)
        progress["step_status"][1] = "success"

        # step3
        progress["current_index"] = 2
        progress["step_status"][2] = "running"
        time.sleep(2)
        progress["step_status"][2] = "success"

        # step4
        progress["current_index"] = 3
        progress["step_status"][3] = "running"
        time.sleep(2)
        progress["step_status"][3] = "success"

        progress["done"] = True
        progress["status"] = "finished"

    except Exception:
        idx = progress.get("current_index", 0)
        progress["step_status"][idx] = "error"
        progress["error"] = True
        progress["status"] = "error"

# ----------------------------
# 啟動流程
# ----------------------------
def start_process():
    global process, progress
    # 如果有殘留的，先砍掉
    if process and process.is_alive():
        process.terminate()  ## 強制中止
        process.join()

    # reset 進度
    progress.clear()
    progress.update(default_progress())

    # 啟動新 process
    process = mp.Process(target=run_report, args=(progress,))
    process.start()

# ----------------------------
# Kill 流程
# ----------------------------
def kill_process():
    global process, progress
    if process and process.is_alive():
        process.terminate()
        process.join()
        progress["status"] = "killed"

# ----------------------------
# Dash App
# ----------------------------
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H3("單一使用者 報告流程 Demo"),
    html.Button("Start Report", id="start-btn", n_clicks=0),
    html.Button("Kill Report", id="kill-btn", n_clicks=0, style={"marginLeft": "10px"}),
    html.Div(id="step-list", style={"marginTop": "20px"}),
    dcc.Interval(id="interval", interval=1000, n_intervals=0, disabled=False)
])

# Start
@app.callback(
    Output("step-list", "children"),
    Input("start-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_report(n):
    start_process()
    return "Report started..."

# Kill
@app.callback(
    Output("step-list", "children", allow_duplicate=True),
    Input("kill-btn", "n_clicks"),
    prevent_initial_call=True
)
def kill_report(n):
    kill_process()
    return "Report killed."

# 更新 UI
@app.callback(
    Output("step-list", "children", allow_duplicate=True),
    Input("interval", "n_intervals"),
    prevent_initial_call=True
)
def update_progress(n):
    p = dict(progress) # 先把共享的進度資料複製一份到本地，後面迴圈就不會反覆跨 process 讀取
    elements = []
    for i, step in enumerate(step_names):
        status = p["step_status"][i] if "step_status" in p else "pending"
        color = {"pending": "gray", "running": "orange", "success": "green", "error": "red"}.get(status, "gray")
        symbol = {"pending": "□", "running": "▶", "success": "✔", "error": "❌"}.get(status, "□")
        elements.append(
            html.Div(f"{symbol} {step}", style={"color": color, "marginBottom": "5px"})
        )
    return elements


if __name__ == "__main__":
    app.run_server(debug=True, threaded=False)


## 緩慢中止 set()

1. Kill 按鈕 → stop_event.set()

2. Step 中呼叫 check_stop(stop_event) → 發現 flag 被設置 → 拋出 Exception

3. Exception 被 run_report_process() 捕捉 → 更新 progress['error'] → 流程結束

也就是說 流程本身要有這個檢查機制，才能「溫和中止」。沒有檢查的話，即使 stop_event.set()，程式仍會跑完當前 step。

In [ ]:
import dash
from dash import html, dcc, Input, Output, State
import multiprocessing
import time

# =========================
# 報告流程 step 與進度
# =========================
step_names = ["撈取資料", "進行 Rule Check", "繪製圖片", "上拋資料"]

def default_progress():
    return {
        'current_index': -1,
        'step_status': ['pending'] * len(step_names),
        'step_times': [None] * len(step_names),
        'done': False,
        'error': False
    }

def check_stop(stop_event):
    if stop_event.is_set():
        raise Exception("中止流程")

def step1(stop_event):
    check_stop(stop_event)
    time.sleep(2)
    return "a"

def step2(a, stop_event):
    check_stop(stop_event)
    time.sleep(2)
    return "b", "c"

def step3(b, c, stop_event):
    check_stop(stop_event)
    time.sleep(2)
    return "d"

def step4(d, stop_event):
    check_stop(stop_event)
    time.sleep(2)
    return "e"

def run_report_process(progress, stop_event):
    try:
        start_time = time.time()
        # Step 1
        progress['current_index'] = 0
        progress['step_status'][0] = 'running'
        a = step1(stop_event)
        progress['step_times'][0] = int(time.time() - start_time)
        progress['step_status'][0] = 'success'

        # Step 2
        progress['current_index'] = 1
        progress['step_status'][1] = 'running'
        b, c = step2(a, stop_event)
        progress['step_times'][1] = int(time.time() - start_time - progress['step_times'][0])
        progress['step_status'][1] = 'success'

        # Step 3
        progress['current_index'] = 2
        progress['step_status'][2] = 'running'
        d = step3(b, c, stop_event)
        progress['step_times'][2] = int(time.time() - start_time - sum(progress['step_times'][:2]))
        progress['step_status'][2] = 'success'

        # Step 4
        progress['current_index'] = 3
        progress['step_status'][3] = 'running'
        e = step4(d, stop_event)
        progress['step_times'][3] = int(time.time() - start_time - sum(progress['step_times'][:3]))
        progress['step_status'][3] = 'success'

        progress['done'] = True
    except Exception:
        progress['step_status'][progress['current_index']] = 'error'
        progress['error'] = True

# =========================
# Dash App
# =========================
app = dash.Dash(__name__)
server = app.server

# 全域 process + stop_event
report_process = None
stop_event = None
manager = multiprocessing.Manager()
progress = manager.dict(default_progress())

app.layout = html.Div([
    html.H3("報告流程 (單使用者，multiprocessing)"),
    html.Button("開始報告", id="run-btn"),
    html.Button("Kill", id="kill-btn", style={"marginLeft": "10px"}),
    html.Div(id="step-list", style={"marginTop": "20px", "fontSize": "16px"}),
    dcc.Interval(id="interval", interval=1000, n_intervals=0, disabled=True)
])

# =========================
# 開始報告
# =========================
@app.callback(
    Output("run-btn", "disabled"),
    Output("interval", "disabled"),
    Input("run-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_report(n):
    global report_process, stop_event, progress
    # 若已有 process 存在，先終止
    if report_process and report_process.is_alive():
        stop_event.set()
        report_process.join()

    # 重設 stop_event 與 progress
    stop_event = multiprocessing.Event()
    progress.clear()
    progress.update(default_progress())

    report_process = multiprocessing.Process(target=run_report_process, args=(progress, stop_event))
    report_process.start()

    return True, False

# =========================
# Kill 報告
# =========================
@app.callback(
    Output("run-btn", "disabled", allow_duplicate=True),
    Output("interval", "disabled", allow_duplicate=True),
    Input("kill-btn", "n_clicks"),
    prevent_initial_call=True
)
def kill_report(n):
    global report_process, stop_event
    if stop_event:
        stop_event.set()
    if report_process and report_process.is_alive():
        report_process.join()
    return False, True

# =========================
# 更新 UI
# =========================
@app.callback(
    Output("step-list", "children"),
    Input("interval", "n_intervals")
)
def update_ui(n):
    elements = []
    for i, step in enumerate(step_names):
        status = progress.get('step_status', ['pending']*len(step_names))[i]
        color = {'pending':'gray','running':'orange','success':'green','error':'red'}.get(status,'gray')
        symbol = {'pending':'□','running':'▶','success':'✔','error':'❌'}.get(status,'□')
        time_spent = f" ({progress.get('step_times', [None]*len(step_names))[i]}秒)" if progress.get('step_times')[i] else ""
        elements.append(html.Div(f"{symbol} {step}{time_spent}", style={"color": color, "marginBottom": "5px"}))
    return elements

if __name__ == "__main__":
    app.run(debug=True, threaded=False)
